In [2]:
from ultralytics import YOLO

model = YOLO("HeadDetec.pt")

results = model.predict(source="1", conf=0.75, show=True, stream = True)

for result in results:
    if len(result.boxes) > 0:
        print(result.boxes[0].xyxy)


1/1: 1... Success  (inf frames of shape 640x480 at 30.00 FPS)

0: 480x640 (no detections), 113.0ms
0: 480x640 (no detections), 62.0ms
0: 480x640 (no detections), 47.0ms
0: 480x640 (no detections), 47.0ms
0: 480x640 (no detections), 48.0ms
0: 480x640 (no detections), 46.0ms
0: 480x640 (no detections), 46.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 47.0ms
0: 480x640 (no detections), 46.0ms
0: 480x640 (no detections), 43.0ms
0: 480x640 (no detections), 43.0ms
0: 480x640 (no detections), 46.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 44.0ms
0: 480x640 (no detections), 44.0ms
0: 480x640 (no detections), 44.0ms
0: 480x640 (no detections), 43.0ms
0: 480x640 (no detections), 42.0ms
0: 480x640 (no detections), 43.0ms
0: 480x640 (no detections), 45.0ms
0: 480x640 (no detections), 50.0ms
0: 480x640 (no detections), 50.0ms
0: 480x640 (no detections

KeyboardInterrupt: 

: 